#### Library Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cartopy.crs as ccrs
from windrose import WindroseAxes
from scipy.stats import weibull_min
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import glob

#### Function Declarations

In [ ]:
def read_netcdf(path):
    data = nc.Dataset(path, 'r')
    return data

In [ ]:
# Recursive function for printing the netCDF file structure
def print_netcdf_structure(data, indent=0):
    indent_str = ' ' * indent
    
    # Print group name if it's a subgroup (indent > 0)
    if indent > 0:
        print(f"{indent_str}Group: {data.path}")
    
    # Print variables in the current group
    for var_name in data.variables:
        var = data.variables[var_name]
        print(f"{indent_str}Variable: {var_name}, shape={var.shape}, dtype={var.dtype}")
    
    # Recursively print subgroups
    for group_name in data.groups:
        print_netcdf_structure(data.groups[group_name], indent + 4)

#### Data Initialisation

In [ ]:
buoy2_data_path = 'data/measurements/2023-11-09_Buoy2_BSH_N-9.nc'
buoy6_data_path = 'data/measurements/2023-11-06_Buoy6_BSH_N-9.nc'
buoy2_data = read_netcdf(buoy2_data_path)
buoy6_data = read_netcdf(buoy6_data_path)

In [ ]:
# Print the structure of the netCDF file
# print("NetCDF File Structure:")
# print_netcdf_structure(buoy2_data)

def print_variable_info(data, variable_name):
    """
    Prints detailed information about a specified variable in a netCDF dataset.
    """
    try:
        var = data.variables[variable_name]
        print(f"Variable Name: {variable_name}")
        print(f"Shape: {var.shape}")
        print(f"Data Type: {var.dtype}")
        print("Attributes:")
        for attr_name in var.ncattrs():
            print(f"  {attr_name}: {var.getncattr(attr_name)}")
    except KeyError:
        print(f"Variable '{variable_name}' not found in the dataset.")        
print_variable_info(buoy2_data, 'time')

time_data = buoy2_data.variables['time'][:]
time_data_converted = pd.to_datetime(time_data, unit='h', origin=pd.Timestamp('1900-01-01'))
print(time_data_converted)



## NetCDF File Structure

- **Variable**: `longitude`, shape=(1,), dtype=float32 ✅
- **Variable**: `latitude`, shape=(1,), dtype=float32 ✅
- **Variable**: `time`, shape=(67680,), dtype=float64 ✅
- **Variable**: `height`, shape=(6,), dtype=int32 ✅

### Group: /ZX_LIDAR_WLBZ_2 
- **Variable**: `wind_speed`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_from_direction`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_from_direction_availability`, shape=(67680, 1, 1, 6), dtype=float64
- **Variable**: `wind_speed_availability`, shape=(67680, 1, 1, 6), dtype=float64
- **Variable**: `wind_speed_min`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_speed_max`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_speed_stddev`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_count`, shape=(67680, 1, 1, 6), dtype=float64
- **Variable**: `turbulence_intensity`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `algorithm_bitmask`, shape=(67680, 1, 1, 6), dtype=float64

### Group: /METEO_WLBZ_2
#### Subgroup: /METEO_WLBZ_2/Vaisala Weather Station
- **Variable**: `rain_accumulation`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `rain_accumulation_calculated`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `rain_duration`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `rain_intensity`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `hail_accumulation`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `hail_duration`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `hail_intensity`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `precipitation_count`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `relative_humidity`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_density`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_density_moist`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `PTH_count`, shape=(67680, 1, 1), dtype=float64
- **Variable**: `wind_speed_average`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_min`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_max`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_count`, shape=(67680, 1, 1), dtype=float64

#### Subgroup: /METEO_WLBZ_2/Airmar Weather Station
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_density`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_min`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_max`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_speed_average`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `wind_from_direction`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `count`, shape=(67680, 1, 1), dtype=float64

### Group: /ADD_DATA_WLBZ_2 ❌
#### Subgroup: /ADD_DATA_WLBZ_2/CTD ❌
- **Variable**: `sea_water_pressure`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_temperature`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_electrical_conductivity`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_salinity`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `speed_of_sound_in_sea_water`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_water_sigma_t`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `count`, shape=(67680, 1, 1), dtype=float64 ❌

#### Subgroup: /ADD_DATA_WLBZ_2/DEFI ❌
- **Variable**: `sea_water_temperature`, shape=(67680, 1, 1), dtype=float64 ❌

#### Subgroup: /ADD_DATA_WLBZ_2/Coda Octopus ❌
- **Variable**: `sea_surface_wave_significant_height`, shape=(67680, 1, 1), dtype=float64 ❌
- **Variable**: `sea_surface_wave_period_at_variance_spectral_density_maximum`, shape=(67680, 1, 1), dtype=float64 ❌

### Group: /ZX_LIDAR_WLBZ_6_MCP ✅
- **Variable**: `wind_from_direction`, shape=(67680, 1, 1, 6), dtype=float64 ✅
- **Variable**: `wind_speed`, shape=(67680, 1, 1, 6), dtype=float64 ✅

### Group: /METEO_WLBZ_6_MCP ✅
#### Subgroup: /METEO_WLBZ_6_MCP/Airmar Weather Station ✅
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅

#### Subgroup: /METEO_WLBZ_6_MCP/Vaisala Weather Station ✅
- **Variable**: `air_pressure`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `air_temperature`, shape=(67680, 1, 1), dtype=float64 ✅
- **Variable**: `relative_humidity`, shape=(67680, 1, 1), dtype=float64 ✅

In [ ]:
# MCP Windspeed Plot

time2 = buoy2_data.variables['time'][:]
windspeed_mcp_buoy2 = buoy2_data.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_speed'][:]
windspeed2 = buoy2_data.groups['ZX_LIDAR_WLBZ_2'].variables['wind_speed'][:]

time6 = buoy6_data.variables['time'][:]
windspeed_mcp_buoy6 = buoy6_data.groups['ZX_LIDAR_WLBZ_2_MCP'].variables['wind_speed'][:]
windspeed6 = buoy6_data.groups['ZX_LIDAR_WLBZ_6'].variables['wind_speed'][:]

fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True, sharex=True)

# axes[0].plot(time2, windspeed_mcp_buoy2[:, 0, 0, 0])
# axes[0].plot(time2, windspeed2[:, 0, 0, 0])
# axes[0].set_title('Buyoy 2')
# axes[0].set_xlabel('Time')
# axes[0].set_ylabel('Wind Speed (m/s)')
# axes[0].legend(['ZX_LIDAR_WLBZ_6_MCP', 'ZX_LIDAR_WLBZ_2'])

axes[0].plot(time6, windspeed_mcp_buoy6[:, 0, 0, 0])
axes[0].plot(time6, windspeed6[:, 0, 0, 0])
axes[0].set_title('Buyoy 6')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Wind Speed (m/s)')
axes[0].legend(['ZX_LIDAR_WLBZ_2_MCP', 'ZX_LIDAR_WLBZ_6'])

axes[1].plot(time2, windspeed_mcp_buoy2[:, 0, 0, 0], linestyle='--', linewidth=0.4)
axes[1].plot(time6, windspeed_mcp_buoy6[:, 0, 0, 0], linestyle='--', linewidth=0.4)
axes[1].set_title('Buyoy 6')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Wind Speed (m/s)')
axes[1].legend(['ZX_LIDAR_WLBZ_6_MCP','ZX_LIDAR_WLBZ_2_MCP'])

### Windroses for Windspeed directions of different Weather Stations

In [ ]:
height = buoy2_data.variables['height'][:]
print("Height:", height)

print("Buoy2 Data")
print(f"Latitude: {buoy2_data.variables['latitude'][:]}")
print(f"Longitude: {buoy2_data.variables['longitude'][:]}")

print("Buoy6 Data")
print(f"Latitude: {buoy6_data.variables['latitude'][:]}")
print(f"Longitude: {buoy6_data.variables['longitude'][:]}")

print(buoy2_data.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_speed'][:].shape)

time = buoy2_data.variables['time'][:]
print(time)




In [ ]:
height_index = 3
chosen_height = buoy2_data.variables['height'][height_index]

# Extract wind direction and speed data from the netCDF file
zx_lidar_wind_dir = buoy2_data.groups['ZX_LIDAR_WLBZ_2'].variables['wind_from_direction'][:,:,:,height_index]
zx_lidar_wind_dir = zx_lidar_wind_dir.reshape(-1)  # Flatten the array
zx_lidar_wind_speed = buoy2_data.groups['ZX_LIDAR_WLBZ_2'].variables['wind_speed'][:,:,:,height_index]

zx_lidar_wind_speed = zx_lidar_wind_speed.reshape(-1)  # Flatten the array

airmar_wind_dir = buoy2_data.groups['METEO_WLBZ_2'].groups['Airmar Weather Station'].variables['wind_from_direction'][:]
airmar_wind_dir = airmar_wind_dir.reshape(-1)  # Flatten the array
airmar_wind_speed = buoy2_data.groups['METEO_WLBZ_2'].groups['Airmar Weather Station'].variables['wind_speed_average'][:]
airmar_wind_speed = airmar_wind_speed.reshape(-1)  # Flatten the array

zx_lidar_mcp_wind_dir = buoy2_data.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_from_direction'][:,:,:,height_index]
zx_lidar_mcp_wind_dir = zx_lidar_mcp_wind_dir.reshape(-1)  # Flatten the array
zx_lidar_mcp_wind_speed = buoy2_data.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_speed'][:,:,:,height_index]

zx_lidar_mcp_wind_speed = zx_lidar_mcp_wind_speed.reshape(-1)  # Flatten the array

# Create a figure with subplots for each wind rose
fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw=dict(projection='windrose'))

# Plot wind rose for ZX_LIDAR_WLBZ_2
ax = axes[0]
ax.bar(zx_lidar_wind_dir, zx_lidar_wind_speed, normed=True, opening=0.8, edgecolor='white', bins=np.arange(0, 22, 4))
ax.set_legend(title="Wind Speed (m/s)")
ax.set_title('ZX_LIDAR_WLBZ_2 ' + str(chosen_height) + 'm')

# Plot wind rose for Airmar Weather Station
ax = axes[1]
ax.bar(airmar_wind_dir, airmar_wind_speed, normed=True, opening=0.8, edgecolor='white', bins=np.arange(0, 22, 4))
ax.set_legend(title="Wind Speed (m/s)")
ax.set_title('METEO_WLBZ_2/Airmar Weather Station')

# Plot wind rose for ZX_LIDAR_WLBZ_6_MCP
ax = axes[2]
ax.bar(zx_lidar_mcp_wind_dir, zx_lidar_mcp_wind_speed, normed=True, opening=0.8, edgecolor='white', bins=np.arange(0, 22, 4))
ax.set_legend(title="Wind Speed (m/s)")
ax.set_title('ZX_LIDAR_WLBZ_6_MCP ' + str(chosen_height) + 'm')
plt.tight_layout()
plt.show()

### Weibull distribution of windspeeds for each weather station

In [ ]:
# Extract wind speed data from the netCDF file
zx_lidar_wind_speed = buoy2_data.groups['ZX_LIDAR_WLBZ_2'].variables['wind_speed'][:,:,:,height_index]
zx_lidar_wind_speed = zx_lidar_wind_speed.reshape(-1)  # Flatten the array

vaisala_wind_speed = buoy2_data.groups['METEO_WLBZ_2'].groups['Vaisala Weather Station'].variables['wind_speed_average'][:]
vaisala_wind_speed = vaisala_wind_speed.reshape(-1)

airmar_wind_speed = buoy2_data.groups['METEO_WLBZ_2'].groups['Airmar Weather Station'].variables['wind_speed_average'][:]
airmar_wind_speed = airmar_wind_speed.reshape(-1)  # Flatten the array

zx_lidar_mcp_wind_speed = buoy2_data.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_speed'][:,:,:,height_index]
zx_lidar_mcp_wind_speed = zx_lidar_mcp_wind_speed.reshape(-1)  # Flatten the array

# Remove non-finite values from the wind speed data
zx_lidar_wind_speed = zx_lidar_wind_speed[np.isfinite(zx_lidar_wind_speed)]
vaisala_wind_speed = vaisala_wind_speed[np.isfinite(vaisala_wind_speed)]
airmar_wind_speed = airmar_wind_speed[np.isfinite(airmar_wind_speed)]
zx_lidar_mcp_wind_speed = zx_lidar_mcp_wind_speed[np.isfinite(zx_lidar_mcp_wind_speed)]

# Create bins for wind speed
bins = np.arange(0, 22, 1)

# Create a figure with subplots for each Weibull distribution
fig, axes = plt.subplots(1, 4, figsize=(15, 5), sharey=True, sharex=True)

# Plot Weibull distribution for ZX_LIDAR_WLBZ_2
ax = axes[0]
hist, _ = np.histogram(zx_lidar_wind_speed, bins=bins, density=True)
x = np.linspace(0, 20, 100)
shape, loc, scale = weibull_min.fit(zx_lidar_wind_speed, floc=0)
ax.plot(x, weibull_min.pdf(x, shape, loc=loc, scale=scale), 'r-', lw=2, label='Weibull')
ax.bar(bins[:-1], hist, width=1, align='edge', alpha=0.5)
ax.set_xlabel('Wind Speed (m/s)')
ax.set_ylabel('Probability Density')
ax.set_title('ZX_LIDAR_WLBZ_2')
ax.legend()

# Plot Weibull distribution for Vaisala Weather Station
ax = axes[1]
hist, _ = np.histogram(vaisala_wind_speed, bins=bins, density=True)
shape, loc, scale = weibull_min.fit(vaisala_wind_speed, floc=0)
ax.plot(x, weibull_min.pdf(x, shape, loc=loc, scale=scale), 'r-', lw=2, label='Weibull')
ax.bar(bins[:-1], hist, width=1, align='edge', alpha=0.5)
ax.set_xlabel('Wind Speed (m/s)')
ax.set_ylabel('Probability Density')
ax.set_title('METEO_WLBZ_2/Vaisala Weather Station')
ax.legend()

# Plot Weibull distribution for Airmar Weather Station
ax = axes[2]
hist, _ = np.histogram(airmar_wind_speed, bins=bins, density=True)
shape, loc, scale = weibull_min.fit(airmar_wind_speed, floc=0)
ax.plot(x, weibull_min.pdf(x, shape, loc=loc, scale=scale), 'r-', lw=2, label='Weibull')
ax.bar(bins[:-1], hist, width=1, align='edge', alpha=0.5)
ax.set_xlabel('Wind Speed (m/s)')
ax.set_ylabel('Probability Density')
ax.set_title('METEO_WLBZ_2/Airmar Weather Station')
ax.legend()

# Plot Weibull distribution for ZX_LIDAR_WLBZ_6_MCP
ax = axes[3]
hist, _ = np.histogram(zx_lidar_mcp_wind_speed, bins=bins, density=True)
shape, loc, scale = weibull_min.fit(zx_lidar_mcp_wind_speed, floc=0)
ax.plot(x, weibull_min.pdf(x, shape, loc=loc, scale=scale), 'r-', lw=2, label='Weibull')
ax.bar(bins[:-1], hist, width=1, align='edge', alpha=0.5)
ax.set_xlabel('Wind Speed (m/s)')
ax.set_ylabel('Probability Density')
ax.set_title('ZX_LIDAR_WLBZ_6_MCP')
ax.legend()

plt.tight_layout()
plt.show()

### Plot timeseries of windspeed including Error

In [ ]:
# Extract time data
time_data = buoy2_data.variables['time'][:]
time_data = pd.to_datetime(time_data, unit='s', origin='unix')

# Extract wind speed data from the netCDF file
zx_lidar_wind_speed = buoy2_data.groups['ZX_LIDAR_WLBZ_2'].variables['wind_speed'][:,:,:,height_index]
zx_lidar_wind_speed = zx_lidar_wind_speed.reshape(-1)  # Flatten the array
zx_lidar_wind_speed_stddev = buoy2_data.groups['ZX_LIDAR_WLBZ_2'].variables['wind_speed_stddev'][:,:,:,height_index]
zx_lidar_wind_speed_stddev = zx_lidar_wind_speed_stddev.reshape(-1)  # Flatten the array

airmar_wind_speed = buoy2_data.groups['METEO_WLBZ_2'].groups['Airmar Weather Station'].variables['wind_speed_average'][:]
airmar_wind_speed = airmar_wind_speed.reshape(-1)  # Flatten the array

zx_lidar_mcp_wind_speed = buoy2_data.groups['ZX_LIDAR_WLBZ_6_MCP'].variables['wind_speed'][:,:,:,height_index]
zx_lidar_mcp_wind_speed = zx_lidar_mcp_wind_speed.reshape(-1)  # Flatten the array


# Create an interactive plot
fig, ax = plt.subplots(figsize=(12, 6))

# Plot wind speed for ZX_LIDAR_WLBZ_2 with error shading
ax.plot(time_data, zx_lidar_wind_speed, label='ZX_LIDAR_WLBZ_2')
ax.fill_between(time_data, zx_lidar_wind_speed - zx_lidar_wind_speed_stddev, zx_lidar_wind_speed + zx_lidar_wind_speed_stddev, alpha=0.2)

# Plot wind speed for Airmar Weather Station
ax.plot(time_data, airmar_wind_speed, label='METEO_WLBZ_2/Airmar Weather Station')

# Plot wind speed for ZX_LIDAR_WLBZ_6_MCP
ax.plot(time_data, zx_lidar_mcp_wind_speed, label='ZX_LIDAR_WLBZ_6_MCP')

ax.set_xlabel('Time')
ax.set_ylabel('Wind Speed (m/s)')
ax.set_title('Wind Speed Time Series')
ax.legend()

# Enable interactive plot
plt.tight_layout()
plt.show()

In [ ]:
def extract_wind_speed(data, group, variable, height_index):
    wind_speed = data.groups[group].variables[variable][:,:,:,height_index]
    wind_speed = wind_speed.reshape(-1)  # Flatten the array
    return wind_speed

def interpolate_lidar_to_era5(era5_df, lidar_df):
    # Set 'Time' as the index for both dataframes
    era5_df.set_index('Time [UTC]', inplace=True)
    lidar_df.set_index('Time', inplace=True)
    
    # Interpolate LIDAR data to match ERA5 timestamps
    lidar_df = lidar_df.reindex(era5_df.index, method='nearest', tolerance=pd.Timedelta('1h'))  # Adjust tolerance as needed
    lidar_df = lidar_df.interpolate()  # Interpolate to fill any gaps after reindexing
    
    # Reset index to merge later if needed
    lidar_df.reset_index(inplace=True)
    era5_df.reset_index(inplace=True)
    
    return lidar_df

def mcp_regression(era5_files, netcdf_path, group, variable, height_index):
    # Read and preprocess ERA5 data
    era5_data = pd.concat([pd.read_csv(f) for f in era5_files], ignore_index=True)
    era5_data['Time [UTC]'] = pd.to_datetime(era5_data['Time [UTC]'])
    era5_data['wind_speed_100m'] = np.sqrt(era5_data['u100']**2 + era5_data['v100']**2)
    
    # Load netCDF data
    lidar_data = read_netcdf(netcdf_path)
    
    # Extract wind speed from LIDAR data
    lidar_wind_speed = extract_wind_speed(lidar_data, group, variable, height_index)
    lidar_df = pd.DataFrame({
        'Time': pd.to_datetime(lidar_data.variables['time'][:], unit='s', origin='unix'),
        'Lidar_Wind_Speed': lidar_wind_speed
    })
    
    # Interpolate or merge flexibly
    lidar_df = interpolate_lidar_to_era5(era5_data, lidar_df)
    
    # Match timestamps
    time_data = pd.to_datetime(lidar_data.variables['time'][:], unit='s', origin='unix')
    lidar_df = pd.DataFrame({
        'Time': time_data,
        'Lidar_Wind_Speed': lidar_wind_speed
    })
    
    # Merge datasets on the nearest time
    merged_data = pd.merge_asof(era5_data.sort_values('Time [UTC]'), lidar_df.sort_values('Time'), left_on='Time [UTC]', right_on='Time', direction='nearest')
    
    # Drop rows with NaN values
    merged_data.dropna(subset=['Lidar_Wind_Speed', 'wind_speed_100m'], inplace=True)
    
    # Check if data is still available
    if merged_data.empty:
        raise ValueError("No data available after preprocessing. Check your data and methods.")
    
        
    # Linear regression
    model = LinearRegression()
    model.fit(merged_data[['Lidar_Wind_Speed']], merged_data['wind_speed_100m'])
    
    # Predict long-term wind speed
    merged_data['Predicted_Wind_Speed'] = model.predict(merged_data[['Lidar_Wind_Speed']])
    
    return merged_data, model

# Usage
era5_files = glob.glob('data/reanalysis/*.csv')
netcdf_path = 'data/measurements/2023-11-09_Buoy2_BSH_N-9.nc'
group = 'ZX_LIDAR_WLBZ_2'
variable = 'wind_speed'
height_index = 2  # Which one

predicted_data, regression_model = mcp_regression(era5_files, netcdf_path, group, variable, height_index)
print(predicted_data.head())
print("Regression coefficients:", regression_model.coef_)